# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide. 

Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords



In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Iron Hack/Week_8/Day_3/Morning/lab-nlp/your-code/archive (1).zip", 
                   error_bad_lines = False, encoding = 'ISO-8859-1')
data.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
data.columns = ['target','id','date','flag','user','text']
data = data.sample(10000).reset_index()

In [ ]:
data

,index,target,id,date,flag,user,text
0,1328502,4,2015528833,Wed Jun 03 04:54:08 PDT 2009,NO_QUERY,gGgceo,Anyhow...check out my blog @ givegodgloryblog....
1,928786,4,1759728676,Sun May 10 19:51:22 PDT 2009,NO_QUERY,NecroMancer666,@scifitv That's heaps cool
2,641110,0,2235158674,Fri Jun 19 00:33:32 PDT 2009,NO_QUERY,robbjbrowning,Morning! Lovely day...and yet I'm in work agai...
3,310464,0,2001106744,Mon Jun 01 23:51:41 PDT 2009,NO_QUERY,Joooooe,Another Cracking Day N A Stuck In This Shity C...
4,716815,0,2259810234,Sat Jun 20 18:11:57 PDT 2009,NO_QUERY,LLLouisa,had a fantastic morning! gotta do some hill tr...
...,...,...,...,...,...,...,...
9995,246302,0,1982213750,Sun May 31 10:45:26 PDT 2009,NO_QUERY,Geo1234,i have let n-philes down
9996,806175,4,1468772266,Tue Apr 07 03:39:13 PDT 2009,NO_QUERY,LadyFLuffstar,also also... stupid lamb.
9997,1558518,4,2185848308,Mon Jun 15 17:59:46 PDT 2009,NO_QUERY,mattcasey,@maizey12 Ha! My blog never got me any offers...
9998,1150019,4,1978406667,Sat May 30 23:15:45 PDT 2009,NO_QUERY,ne1else,my poor little mixer is getting broken in real...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [ ]:
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """

    s = s.lower()
    # Replace urls by ''
    s = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ' , s) 
    # Replace html tags by ''
    s = BeautifulSoup(s, 'lxml').get_text().strip()
    # Replace e-mails by ''
    s = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)'," ", s) 
    s = re.sub(r'\W+',' ',s) # Replace everything non-alpahnumeric by ' '
    s = re.sub(r'\s+',' ',s) # Replace one or more whitespaces by  ' '
    s = re.sub(r'\d+',' ',s) # Replace one or more digits by  ' '

    return s

In [ ]:
def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return word_tokenize(s)

In [ ]:
def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = PorterStemmer()
    lem = WordNetLemmatizer()
    stem = (ps.stem(x) for x in l)
    stem_lem = (lem.lemmatize(x) for x in stem)
    return stem_lem


In [ ]:
def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = list(stopwords.words('english')) 
    clean_list = [word for word in l if word not in stop_words]

    return clean_list

In [ ]:
data['text_processed']= data['text'].apply(clean_up)
data['text_processed'] = data['text_processed'].apply(tokenize)
data['text_processed'] = data['text_processed'].apply(stem_and_lemmatize)
data['text_processed'] = data['text_processed'].apply(remove_stopwords)


In [ ]:
data.dtypes

index              int64
target             int64
id                 int64
date              object
flag              object
user              object
text              object
text_processed    object
dtype: object

In [ ]:

data.head()

,index,target,id,date,flag,user,text,text_processed
0,1328502,4,2015528833,Wed Jun 03 04:54:08 PDT 2009,NO_QUERY,gGgceo,Anyhow...check out my blog @ givegodgloryblog....,"[anyhow, check, blog, givegodgloryblog, com, i..."
1,928786,4,1759728676,Sun May 10 19:51:22 PDT 2009,NO_QUERY,NecroMancer666,@scifitv That's heaps cool,"[scifitv, heap, cool]"
2,641110,0,2235158674,Fri Jun 19 00:33:32 PDT 2009,NO_QUERY,robbjbrowning,Morning! Lovely day...and yet I'm in work agai...,"[morn, love, day, yet, work, today]"
3,310464,0,2001106744,Mon Jun 01 23:51:41 PDT 2009,NO_QUERY,Joooooe,Another Cracking Day N A Stuck In This Shity C...,"[anoth, crack, day, n, stuck, thi, shiti, coll..."
4,716815,0,2259810234,Sat Jun 20 18:11:57 PDT 2009,NO_QUERY,LLLouisa,had a fantastic morning! gotta do some hill tr...,"[fantast, morn, got, ta, hill, train, regret, ..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [ ]:
text_words = data['text_processed'].explode().to_list()
text_words

['anyhow',
 'check',
 'blog',
 'givegodgloryblog',
 'com',
 'inspir',
 'challeng',
 'buy',
 'shirt',
 'thx',
 'scifitv',
 'heap',
 'cool',
 'morn',
 'love',
 'day',
 'yet',
 'work',
 'today',
 'anoth',
 'crack',
 'day',
 'n',
 'stuck',
 'thi',
 'shiti',
 'colleg',
 'onli',
 'gd',
 'news',
 'scotrail',
 'actual',
 'got',
 'today',
 'lol',
 'fantast',
 'morn',
 'got',
 'ta',
 'hill',
 'train',
 'regret',
 'go',
 'tour',
 'de',
 'franc',
 'retardedly_cut',
 'cheer',
 'everyon',
 'ok',
 'think',
 'figur',
 'first',
 'clue',
 'ie',
 'comp',
 'none',
 'wiser',
 'cash',
 'though',
 'awesom',
 'work',
 'morn',
 'paint',
 'even',
 'luvsebbiiecak',
 'lol',
 'wan',
 'na',
 'share',
 'updat',
 'get',
 'stevecarti',
 'well',
 'deserv',
 'photo',
 'love',
 'haunt',
 'definit',
 'pick',
 'book',
 'photo',
 'cake',
 'rockstar',
 'chrisboudi',
 'rb',
 'gh',
 'katamari',
 'mine',
 'brother',
 'xbox',
 'kept',
 'ish',
 'oh',
 'yeah',
 'gh',
 'p',
 'castiel',
 'prick',
 'get',
 'supernatur',
 'angel',
 'a

In [ ]:
corpus =  nltk.FreqDist(text_words)
corpus

FreqDist({'go': 823, 'day': 723, 'wa': 669, 'get': 667, 'thi': 596, 'good': 550, 'work': 528, 'like': 496, 'love': 488, 'u': 452, ...})

In [ ]:
corpus = sorted(corpus.items(), key=lambda x:x[1], reverse=True)
corpus

[('go', 823),
 ('day', 723),
 ('wa', 669),
 ('get', 667),
 ('thi', 596),
 ('good', 550),
 ('work', 528),
 ('like', 496),
 ('love', 488),
 ('u', 452),
 ('today', 436),
 ('got', 428),
 ('thank', 399),
 ('lol', 383),
 ('time', 379),
 ('miss', 373),
 ('one', 370),
 ('im', 367),
 ('back', 351),
 ('know', 334),
 ('want', 324),
 ('see', 317),
 ('feel', 314),
 ('think', 311),
 ('make', 307),
 ('watch', 287),
 ('realli', 280),
 ('need', 279),
 ('na', 276),
 ('new', 276),
 ('hope', 276),
 ('well', 275),
 ('still', 275),
 ('home', 264),
 ('night', 252),
 ('come', 247),
 ('oh', 241),
 ('look', 235),
 ('wait', 226),
 ('ha', 224),
 ('much', 219),
 ('twitter', 219),
 ('last', 217),
 ('morn', 214),
 ('tomorrow', 213),
 ('haha', 212),
 ('great', 201),
 ('sleep', 200),
 ('wish', 200),
 ('fun', 195),
 ('whi', 194),
 ('thing', 193),
 ('onli', 188),
 ('sad', 186),
 ('sorri', 181),
 ('follow', 175),
 ('veri', 175),
 ('bad', 167),
 ('would', 164),
 ('happi', 164),
 ('even', 163),
 ('tonight', 163),
 ('tri', 

In [ ]:
corpus_freq = [word[0] for word in corpus[:3001]] #Top 3000 words
corpus_freq = corpus_freq[0:] 

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [ ]:
def find_features(text):
  words = set(text)
  features = {}
  for w in corpus:
    features[w] = (w in words)
  return features

In [ ]:
feature_set = [(find_features(text), bool(target)) for (text, target) in zip(data['text_processed'], data['target'])]
feature_set

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [ ]:
# your code here

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here